In [2]:
#Atur Timezone
%pip install pytz

#Untuk melakukan Geocoding
%pip install geopy

Note: you may need to restart the kernel to use updated packages.

   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]

Note: you may need to restart the kernel to use updated packages.


In [14]:
#Standard Library
import pandas as pd
import pandas_gbq as gbq
import numpy as np
import requests

#DateTime and TimeZone Library
from datetime import datetime, timezone
from pytz import timezone as tz

#Geocoding Library
from geopy.geocoders import Nominatim

#Third Party Import
from google.oauth2 import service_account 

#Logging
import logging
#setup logger
logging.basicConfig(level=logging.INFO)

Table of Content

    1. Extract

    2. Transform

    3. Load

## Data Extraction

In [17]:
#Define BASE_URL

#BASE URL for real-time earthquake data (past 24 hours)
BASE_URL = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson'

# BASE URL for histerical earthquake data
BASE_URL_1 = "https://earthquake.usgs.gov/earthquakes/fdsnws/event/1/query"

In [ ]:
#Get All Real-Time Data Earthquake

response = requests.get(BASE_URL)
data = response.json()
data

{'generated': 1752031806000,
 'url': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson',
 'title': 'USGS All Earthquakes, Past Day',
 'status': 200,
 'api': '1.14.1',
 'count': 451}

In [22]:
records = []

for feature in data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']  # [lon, lat, depth]

    record = {
        "id": feature.get("id"),
        "place": props.get("place"),
        "mag": props.get("mag"),
        "time": pd.to_datetime(props.get("time"), unit='ms'),
        "updated": pd.to_datetime(props.get("updated"), unit='ms'),
        "tz": props.get("tz"),
        "felt": props.get("felt"),
        "cdi": props.get("cdi"),
        "mmi": props.get("mmi"),
        "alert": props.get("alert"),
        "status": props.get("status"),
        "tsunami": props.get("tsunami"),
        "sig": props.get("sig"),
        "net": props.get("net"),
        "code": props.get("code"),
        "ids": props.get("ids"),
        "sources": props.get("sources"),
        "types": props.get("types"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth": coords[2],
        "fetched_at": datetime.now(timezone.utc)
    }
    records.append(record)


records_df = pd.DataFrame(records)

In [23]:
records_df


,id,place,mag,time,updated,tz,felt,cdi,mmi,alert,...,sig,net,code,ids,sources,types,longitude,latitude,depth,fetched_at
0,nc75207327,"2 km E of Pinnacles, CA",1.87,2025-07-09 03:27:19.310,2025-07-09 03:28:54.752,None,NaN,NaN,NaN,None,...,54,nc,75207327,",nc75207327,",",nc,",",nearby-cities,origin,phase-data,",-121.122833,36.535999,9.200000,2025-07-09 03:43:11.213877+00:00
1,nc75207322,"8 km NNW of The Geysers, CA",0.68,2025-07-09 03:22:44.550,2025-07-09 03:24:19.610,None,NaN,NaN,NaN,None,...,7,nc,75207322,",nc75207322,",",nc,",",nearby-cities,origin,phase-data,",-122.815002,38.828499,1.410000,2025-07-09 03:43:11.214300+00:00
2,uw62136042,"22 km ENE of Ashford, Washington",0.36,2025-07-09 03:17:23.080,2025-07-09 03:18:56.910,None,NaN,NaN,NaN,None,...,2,uw,62136042,",uw62136042,",",uw,",",origin,phase-data,",-121.768166,46.846832,-1.720000,2025-07-09 03:43:11.214865+00:00
3,nc75207317,"4 km NNW of The Geysers, CA",0.75,2025-07-09 03:17:09.340,2025-07-09 03:18:44.596,None,NaN,NaN,NaN,None,...,9,nc,75207317,",nc75207317,",",nc,",",nearby-cities,origin,phase-data,",-122.778999,38.805168,2.370000,2025-07-09 03:43:11.215281+00:00
4,ok2025nhoo,"5 km E of Meridian, Oklahoma",1.62,2025-07-09 03:16:14.009,2025-07-09 03:20:12.679,None,NaN,NaN,NaN,None,...,40,ok,2025nhoo,",ok2025nhoo,",",ok,",",origin,phase-data,",-97.184959,35.847298,7.242111,2025-07-09 03:43:11.215819+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,av93661711,"7 km W of Akutan, Alaska",0.64,2025-07-08 04:10:43.050,2025-07-08 21:22:40.670,None,NaN,NaN,NaN,None,...,6,av,93661711,",av93661711,",",av,",",origin,phase-data,",-165.888333,54.143000,4.810000,2025-07-09 03:43:11.324078+00:00
447,us7000qba0,"29 km NNW of Diphu, India",4.20,2025-07-08 03:52:10.603,2025-07-08 04:08:33.040,None,NaN,NaN,NaN,None,...,271,us,7000qba0,",us7000qba0,",",us,",",origin,phase-data,",93.361700,26.102100,10.000000,2025-07-09 03:43:11.324286+00:00
448,nc75206892,"6 km WSW of Gilroy, CA",1.47,2025-07-08 03:52:05.750,2025-07-08 04:12:20.273,None,NaN,NaN,NaN,None,...,33,nc,75206892,",nc75206892,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",-121.622833,36.972668,3.700000,2025-07-09 03:43:11.324491+00:00
449,tx2025nhkbrf,"37 km ESE of Malaga, New Mexico",1.00,2025-07-08 03:47:39.121,2025-07-08 23:58:38.906,None,NaN,NaN,NaN,None,...,15,tx,2025nhkbrf,",tx2025nhkbrf,",",tx,",",origin,phase-data,",-103.725000,32.066000,7.984000,2025-07-09 03:43:11.324697+00:00


# 2. Transformation

- Basic Cleaning
- Enlistment

In [24]:
df = records_df.copy()

In [25]:
#Drop duplicate Records
df = df.drop_duplicates(subset=['id'])

#Drop rows with null magnitude or location
df = df.dropna(subset = ["mag", "place", "longitude", "latitude", "time"])

In [26]:
# Convert time columns to datetime if not already
# Convert UTC time columns to datetime with Asia/Jakarta timezone
WIB = tz('Asia/Jakarta')
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['updated'] = pd.to_datetime(df['updated'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['fetched_at'] = pd.to_datetime(df['fetched_at'], errors='coerce').dt.tz_convert('UTC').dt.tz_convert(WIB)

In [27]:
#  Filter only valid magnitudes
df = df[df['mag'] >= 0]

# Select relevant columns
selected_columns = [
    "id", "place", "mag", "time", "updated",
    "latitude", "longitude", "depth",
    "tsunami", "sig", "status", "alert", "types",
    "felt", "cdi", "mmi", "fetched_at"
]
df = df[selected_columns]

In [28]:
# 2b. Data Enrichment

# Add Magnitude Category
def classify_magnitude(mag):
        if mag < 2.0:
            return "Micro"
        elif mag < 4.0:
            return "Minor"
        elif mag < 5.5:
            return "Light"
        elif mag < 7.0:
            return "Moderate"
        elif mag < 8.0:
            return "Strong"
        else:
            return "Major"

df['mag_category'] = df['mag'].apply(classify_magnitude)

In [29]:
df["mag_category"]

0      Micro
1      Micro
2      Micro
3      Micro
4      Micro
       ...  
446    Micro
447    Light
448    Micro
449    Micro
450    Micro
Name: mag_category, Length: 421, dtype: object

In [33]:
# Add address details
def get_address_detail(latitude, longitude):
    result = {}
    coder = Nominatim(user_agent="myGeocoder", timeout=4)
    # print(coder.reverse(f"{latitude}, {longitude}"))
    address = coder.reverse(f"{latitude}, {longitude}")

    result["city"] = address.raw.get("address").get("city") if address else np.nan
    result["state"] = address.raw.get("address").get("state") if address else np.nan
    result["country"] = address.raw.get("address").get("country") if address else np.nan
    return result

address_detail = df.apply(lambda row: get_address_detail(row['latitude'], row['longitude']), axis=1, result_type='expand')
df = pd.concat([df, address_detail], axis=1)

In [34]:
# Add time-based columns
df['day_of_week'] = df['time'].dt.day_name()
df['hour_of_day'] = df['time'].dt.hour

In [35]:
# Save data into csv
df.to_csv(f"../data/earthquakes-(datetime.now().strftime('%Y-%m-%d')).csv", index=False)

In [36]:
#Big Query Configuration
PROJECT_ID = "earthquake-jcds-0612-465304"
TABLE_ID = "earthquakes.events" #dataset.id
CREDENTIAL_PATH = "../Config/earthquake-jcds-0612-465304-f5f2bbcf02fa.json"

In [37]:
credentials = service_account.Credentials.from_service_account_file(CREDENTIAL_PATH)

In [39]:
gbq.to_gbq(
    dataframe = df,
    destination_table = TABLE_ID,
    project_id = PROJECT_ID,
    credentials = credentials,
    if_exists = "replace"
)

421 out of 421 rows loaded.<?, ?it/s]INFO:pandas_gbq.gbq_connector:
100%|██████████| 1/1 [00:00<00:00, 1168.00it/s]
